In [2]:
import torchvision
# transform is used to convert data into Tensor form with transformations
import torchvision.transforms as transforms
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

trans = transforms.Compose([
    # To resize image
    transforms.Resize((32,32)),
    transforms.ToTensor(),
    # To normalize image
    transforms.Normalize((0.5,), (0.5,))
])

train_set = torchvision.datasets.MNIST(
root = './data',
train = True,
download = True,
transform = trans
)

test_set = torchvision.datasets.MNIST(
root = './data',
train = False,
download = True,
transform = trans
)

In [4]:
import torch
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data.dataloader import DataLoader

# this is one of Hyper parameter, but let's select given below value
batch_size = 512

from torchvision.utils import make_grid
# this will help us to create Grid of images

import torch.nn as nn
import torch.nn.functional as F

class LeNet5(nn.Module):

    def __init__(self, num_classes):

        super().__init__()

        self.num_classes = num_classes

        self.features = nn.Sequential(
            nn.Conv2d(1, 6, kernel_size = 5),
            nn.Tanh(),
            nn.MaxPool2d(kernel_size = 2),
            nn.Conv2d(6, 16, kernel_size = 5),
            nn.Tanh(),
            nn.MaxPool2d(kernel_size = 2)
        )

        self.classifier = nn.Sequential(
            nn.Linear(16*5*5, 120),
            nn.Tanh(),
            nn.Linear(120, 84),
            nn.Tanh(),
            nn.Linear(84, num_classes)
        )



    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        logit = self.classifier(x)
        return logit

def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')

def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device

    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl:
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

def accuracy(output, labels):
    _, preds = torch.max(output, dim = 1)

    return torch.sum(preds == labels).item() / len(preds)


device = get_default_device()
device

def eevaluate(model, loss_fn, val_dl, metric = None, device='cuda'):

    with torch.no_grad():

        results = [loss_batch(model, loss_fn, x, y, metric = metric) for x, y in val_dl]

        losses, nums, metrics = zip(*results)

        total = np.sum(nums)

        avg_loss = np.sum(np.multiply(losses, nums)) / total

        avg_metric = None

        if metric is not None:
            avg_metric = np.sum(np.multiply(metrics, nums)) / total

    return avg_loss, total, avg_metric

def loss_batch(model, loss_func, x, y, opt = None, metric = None):

    pred = model(x)

    loss = loss_func(pred, y)

    if opt is not None:

        loss.backward()
        opt.step()
        opt.zero_grad()

    metric_result = None

    if metric is not None:

        metric_result = metric(pred, y)

    return loss.item(), len(x), metric_result

In [5]:
model = torch.load('lenet.pth', map_location=device)

In [6]:
test_loader = DeviceDataLoader(DataLoader(test_set, batch_size=256), device)
result = eevaluate(model, F.cross_entropy, test_loader, metric = accuracy)
result
Accuracy = result[2] * 100
Accuracy
loss = result[0]
print("Total Losses: {}, Accuracy: {}".format(loss, Accuracy))

Total Losses: 0.03845255719994893, Accuracy: 98.9


In [8]:
layer_names2 = [name for name in model.state_dict().keys() if 'weight' in name]
num_weights_per_layer2 = {name: model.state_dict()[name].numel() for name in layer_names2}
n = 0
for _ in layer_names2:
  n = n + num_weights_per_layer2[_]
print(n)

61470


In [12]:
import random

repititions = 100
BER = 0.01

print(int(BER * n))
acc = []

count = 0

for i in range(repititions):
  model_copy = torch.load('lenet.pth')
  model_copy = model_copy.to('cuda')
  layer_names = [name for name in model.state_dict().keys() if 'weight' in name]
  num_weights_per_layer = {name: model.state_dict()[name].numel() for name in layer_names}
  state_dict = model_copy.state_dict()
  for _ in range(int(BER * n)):
    layer = random.choice(layer_names)
    index = random.randint(0, num_weights_per_layer[layer] - 1)
    weight = state_dict[layer].view(-1).to('cuda')
        # print(weight)
        # weight[weight_idx] += 0.01  # Perturb the weight slightly
    
    weight[index] = weight[index] + 0.1
  model_copy.load_state_dict(state_dict)

    # Evaluate the perturbed model on a validation set
  model_copy.eval()
  result = eevaluate(model_copy, F.cross_entropy, test_loader, metric = accuracy)
  print(count)
  print("accuracy:", result[2] * 100)
  acc.append(result[2] * 100)
  count += 1

avg_acc = sum(acc)/len(acc)
print("average accuracy=", avg_acc)
print("accuracy drop=", 98.9 - avg_acc)

614
0
accuracy: 81.3
1
accuracy: 83.26
2
accuracy: 85.35000000000001
3
accuracy: 84.15
4
accuracy: 93.67
5
accuracy: 85.18
6
accuracy: 80.36
7
accuracy: 88.12
8
accuracy: 83.6
9
accuracy: 78.5
10
accuracy: 88.67
11
accuracy: 73.75
12
accuracy: 90.61
13
accuracy: 84.94
14
accuracy: 85.06
15
accuracy: 74.15
16
accuracy: 90.11
17
accuracy: 85.81
18
accuracy: 83.64
19
accuracy: 80.28
20
accuracy: 80.36
21
accuracy: 74.57000000000001
22
accuracy: 77.42
23
accuracy: 78.3
24
accuracy: 84.39
25
accuracy: 86.02
26
accuracy: 85.85000000000001
27
accuracy: 88.2
28
accuracy: 79.43
29
accuracy: 73.99
30
accuracy: 85.53
31
accuracy: 75.39
32
accuracy: 84.50999999999999
33
accuracy: 88.72
34
accuracy: 86.78
35
accuracy: 80.32000000000001
36
accuracy: 80.13
37
accuracy: 85.57000000000001
38
accuracy: 74.72999999999999
39
accuracy: 73.86
40
accuracy: 74.71
41
accuracy: 80.44
42
accuracy: 89.08
43
accuracy: 87.92
44
accuracy: 76.87
45
accuracy: 86.57000000000001
46
accuracy: 71.43
47
accuracy: 76.5
48
a

In [14]:
import random

repititions = 100
BER = 0.01

print(int(BER * n))
acc = []

count = 0
protected_layer = 'features.0.weight'

for i in range(repititions):
  model_copy = torch.load('lenet.pth')
  model_copy = model_copy.to('cuda')
  layer_names = [name for name in model.state_dict().keys() if 'weight' in name]
  num_weights_per_layer = {name: model.state_dict()[name].numel() for name in layer_names}
  state_dict = model_copy.state_dict()
  for _ in range(int(BER * n)):
    layer = random.choice(layer_names)
    index = random.randint(0, num_weights_per_layer[layer] - 1)
    weight = state_dict[layer].view(-1).to('cuda')
        # print(weight)
        # weight[weight_idx] += 0.01  # Perturb the weight slightly
    if layer == protected_layer:
        # print("protected")
        weight[index] = weight[index]
    else:
        weight[index] = weight[index] + 0.1
  model_copy.load_state_dict(state_dict)

    # Evaluate the perturbed model on a validation set
  model_copy.eval()
  result = eevaluate(model_copy, F.cross_entropy, test_loader, metric = accuracy)
  print(count)
  print("accuracy:", result[2] * 100)
  acc.append(result[2] * 100)
  count += 1

avg_acc = sum(acc)/len(acc)
print("average accuracy=", avg_acc)
print("accuracy drop=", 98.9 - avg_acc)

614
0
accuracy: 98.85000000000001
1
accuracy: 98.81
2
accuracy: 98.81
3
accuracy: 98.81
4
accuracy: 98.82
5
accuracy: 98.83999999999999
6
accuracy: 98.82
7
accuracy: 98.77
8
accuracy: 98.82
9
accuracy: 98.8
10
accuracy: 98.82
11
accuracy: 98.83
12
accuracy: 98.79
13
accuracy: 98.71
14
accuracy: 98.77
15
accuracy: 98.8
16
accuracy: 98.85000000000001
17
accuracy: 98.8
18
accuracy: 98.74000000000001
19
accuracy: 98.78
20
accuracy: 98.79
21
accuracy: 98.91
22
accuracy: 98.8
23
accuracy: 98.75
24
accuracy: 98.83
25
accuracy: 98.74000000000001
26
accuracy: 98.78
27
accuracy: 98.77
28
accuracy: 98.72
29
accuracy: 98.83
30
accuracy: 98.75
31
accuracy: 98.83
32
accuracy: 98.77
33
accuracy: 98.88
34
accuracy: 98.74000000000001
35
accuracy: 98.77
36
accuracy: 98.79
37
accuracy: 98.7
38
accuracy: 98.71
39
accuracy: 98.76
40
accuracy: 98.76
41
accuracy: 98.8
42
accuracy: 98.82
43
accuracy: 98.85000000000001
44
accuracy: 98.83
45
accuracy: 98.7
46
accuracy: 98.85000000000001
47
accuracy: 98.82
48
ac

In [18]:
layer_name = 'features.0.weight'

# Get the number of weights in the specified layer
if layer_name in state_dict:
    num_weights = state_dict[layer_name].numel()  # Get the number of elements in the weight tensor
    print(f"Number of weights in layer '{layer_name}': {num_weights}")
else:
    print(f"Layer '{layer_name}' not found in the model's state_dict.")

Number of weights in layer 'features.0.weight': 150
